In [111]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [74]:
nba_mvp_history = pd.read_csv("Datasets/mvps_stats_all_seasons_revised.csv")
nba_mvp_history.head(10)

,Unnamed: 0,Season,Rank,Player,Pos,Age,G,MP,PTS,AST,...,eFG%,FT%,PF,WS,WS/48,First,Pts Won,Pts Max,Share,MVP
0,0,1996-97,1,Karl Malone,PF,33,82,36.6,27.4,4.5,...,0.550,0.755,2.6,16.7,0.268,63,986,1150,0.857,1
1,1,1996-97,2,Michael Jordan,SG,33,82,37.9,29.6,4.3,...,0.516,0.833,1.9,18.3,0.283,52,957,1150,0.832,0
2,2,1996-97,3,Grant Hill,SF,24,80,39.3,21.4,7.3,...,0.500,0.711,2.3,14.6,0.223,0,376,1150,0.327,0
3,3,1996-97,4,Tim Hardaway,PG,30,81,38.7,20.3,8.6,...,0.489,0.799,2.0,12.9,0.198,0,238,1150,0.207,0
4,4,1996-97,5,Glen Rice,SF,29,79,42.6,26.8,2.0,...,0.546,0.867,2.4,10.8,0.154,0,134,1150,0.117,0
5,5,1996-97,6,Gary Payton,PG,28,82,39.2,21.8,7.1,...,0.517,0.715,2.5,12.9,0.193,0,105,1150,0.091,0
6,6,1996-97,7,Hakeem Olajuwon,C,34,78,36.6,23.2,3.0,...,0.512,0.787,3.2,9.1,0.154,0,95,1150,0.083,0
7,7,1996-97,8,Patrick Ewing,C,34,78,37.0,22.4,2.0,...,0.489,0.754,3.2,9.8,0.163,0,57,1150,0.050,0
8,8,1996-97,9T,Anthony Mason,PF,30,73,43.1,16.2,5.7,...,0.525,0.745,2.8,11.4,0.173,0,7,1150,0.006,0
9,9,1996-97,9T,Shaquille O'Neal,C,24,51,38.1,26.2,3.1,...,0.557,0.484,3.5,8.0,0.197,0,7,1150,0.006,0


In [79]:
mvp = nba_mvp_history.drop(columns = ['Unnamed: 0', 'Rank', 'Season', 'Pos', 'First', 'Pts Won', 'Pts Max', 'Share'])

In [80]:
mvp

,Player,Age,G,MP,PTS,AST,TRB,STL,BLK,FG%,eFG%,FT%,PF,WS,WS/48,MVP
0,Karl Malone,33,82,36.6,27.4,4.5,9.9,1.4,0.6,0.550,0.550,0.755,2.6,16.7,0.268,1
1,Michael Jordan,33,82,37.9,29.6,4.3,5.9,1.7,0.5,0.486,0.516,0.833,1.9,18.3,0.283,0
2,Grant Hill,24,80,39.3,21.4,7.3,9.0,1.8,0.6,0.496,0.500,0.711,2.3,14.6,0.223,0
3,Tim Hardaway,30,81,38.7,20.3,8.6,3.4,1.9,0.1,0.415,0.489,0.799,2.0,12.9,0.198,0
4,Glen Rice,29,79,42.6,26.8,2.0,4.0,0.9,0.3,0.477,0.546,0.867,2.4,10.8,0.154,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,Stephen Curry,34,56,34.7,29.4,6.3,6.1,0.9,0.4,0.493,0.614,0.915,2.1,7.8,0.192,0
394,Jimmy Butler,33,64,33.4,22.9,5.3,5.9,1.8,0.3,0.539,0.560,0.850,1.3,12.3,0.277,0
395,De'Aaron Fox,25,73,33.4,25.0,6.1,4.2,1.1,0.3,0.512,0.557,0.780,2.4,7.4,0.146,0
396,Jalen Brunson,26,68,35.0,24.0,6.2,3.5,0.9,0.2,0.491,0.547,0.829,2.2,8.7,0.175,0


In [91]:
mvp.columns[1:15]

Index(['Age', 'G', 'MP', 'PTS', 'AST', 'TRB', 'STL', 'BLK', 'FG%', 'eFG%',
       'FT%', 'PF', 'WS', 'WS/48'],
      dtype='object')

In [92]:
features = mvp.columns[1:15]

X = mvp[features]
y = mvp['MVP']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [93]:
X

,Age,G,MP,PTS,AST,TRB,STL,BLK,FG%,eFG%,FT%,PF,WS,WS/48
0,33,82,36.6,27.4,4.5,9.9,1.4,0.6,0.550,0.550,0.755,2.6,16.7,0.268
1,33,82,37.9,29.6,4.3,5.9,1.7,0.5,0.486,0.516,0.833,1.9,18.3,0.283
2,24,80,39.3,21.4,7.3,9.0,1.8,0.6,0.496,0.500,0.711,2.3,14.6,0.223
3,30,81,38.7,20.3,8.6,3.4,1.9,0.1,0.415,0.489,0.799,2.0,12.9,0.198
4,29,79,42.6,26.8,2.0,4.0,0.9,0.3,0.477,0.546,0.867,2.4,10.8,0.154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,34,56,34.7,29.4,6.3,6.1,0.9,0.4,0.493,0.614,0.915,2.1,7.8,0.192
394,33,64,33.4,22.9,5.3,5.9,1.8,0.3,0.539,0.560,0.850,1.3,12.3,0.277
395,25,73,33.4,25.0,6.1,4.2,1.1,0.3,0.512,0.557,0.780,2.4,7.4,0.146
396,26,68,35.0,24.0,6.2,3.5,0.9,0.2,0.491,0.547,0.829,2.2,8.7,0.175


In [94]:
y

0      1
1      0
2      0
3      0
4      0
      ..
393    0
394    0
395    0
396    0
397    0
Name: MVP, Length: 398, dtype: int64

In [95]:
lr_model = LogisticRegression()

model = lr_model.fit(X_train, y_train)
model

/Users/lazy_boy/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [104]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy for Logistic Regression Model: {accuracy:.2%}")

Accuracy for Logistic Regression Model: 92.50%


## RandomForestClassifer

In [114]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)

rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [115]:
y_pred2 = rf.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)
print(f"Accuracy for RandomForestClassifier Model: {accuracy2:.2%}")

Accuracy for RandomForestClassifier Model: 92.50%


## Test the Model

In [132]:
test_data = pd.read_csv("Datasets/all_nba_seasons_revised.csv")
test_data.head()

,Unnamed: 0,Season,Player,Pos,Age,G,MP,PTS,AST,TRB,STL,BLK,FG%,eFG%,FT%,PF,WS,WS/48,MVP
0,0,1996-97,Mahmoud Abdul-Rauf,PG,27,75,28.4,13.7,2.5,1.6,0.7,0.1,0.445,0.496,0.846,2.3,2.7,0.060,NaN
1,1,1996-97,Shareef Abdur-Rahim,PF,20,80,35.0,18.7,2.2,6.9,1.0,1.0,0.453,0.456,0.746,2.5,2.9,0.049,NaN
2,3,1996-97,Cory Alexander,PG,23,80,18.2,7.2,3.2,1.5,1.0,0.2,0.396,0.492,0.736,1.9,1.1,0.035,NaN
3,7,1996-97,Ray Allen,SG,21,82,30.9,13.4,2.6,4.0,0.9,0.1,0.430,0.494,0.823,2.7,4.9,0.092,NaN
4,11,1996-97,Kenny Anderson,PG,26,82,37.6,17.5,7.1,4.4,2.0,0.2,0.427,0.485,0.768,2.7,12.4,0.193,NaN


In [133]:
test_data = test_data[test_data["Season"] == '2023-24']
test_data

,Unnamed: 0,Season,Player,Pos,Age,G,MP,PTS,AST,TRB,STL,BLK,FG%,eFG%,FT%,PF,WS,WS/48,MVP
8148,16051,2023-24,Precious Achiuwa,C,24,14,18.1,7.3,2.0,6.1,0.6,0.4,0.432,0.463,0.560,1.8,0.3,0.062,NaN
8149,16052,2023-24,Bam Adebayo,C,26,16,33.6,22.3,3.9,9.9,1.2,1.0,0.522,0.524,0.813,2.4,1.7,0.156,NaN
8150,16053,2023-24,Ochai Agbaji,SG,23,20,20.1,5.8,1.1,2.7,0.6,0.6,0.458,0.578,0.667,1.5,0.5,0.057,NaN
8151,16054,2023-24,Santi Aldama,PF,23,13,27.2,13.9,1.9,6.3,0.8,0.7,0.453,0.525,0.609,1.8,0.5,0.073,NaN
8152,16055,2023-24,Nickeil Alexander-Walker,SG,25,19,23.5,6.7,2.7,2.2,0.9,0.7,0.410,0.521,0.455,1.9,0.8,0.086,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8444,16543,2023-24,Zion Williamson,PF,23,18,31.2,22.7,4.9,5.8,1.2,0.3,0.569,0.572,0.664,2.6,1.7,0.147,NaN
8445,16546,2023-24,James Wiseman,C,22,10,11.1,5.4,0.3,3.2,0.1,0.3,0.649,0.649,0.545,1.8,0.2,0.065,NaN
8446,16547,2023-24,Christian Wood,PF,28,21,20.2,7.0,0.8,5.9,0.2,0.6,0.443,0.504,0.689,1.8,0.3,0.038,NaN
8447,16550,2023-24,Trae Young,PG,25,18,36.0,27.1,10.7,2.9,1.6,0.0,0.409,0.486,0.864,1.5,2.0,0.147,NaN


In [134]:
test_data = test_data.drop(columns = ['Unnamed: 0', 'Pos', 'Season'])
test_data

,Player,Age,G,MP,PTS,AST,TRB,STL,BLK,FG%,eFG%,FT%,PF,WS,WS/48,MVP
8148,Precious Achiuwa,24,14,18.1,7.3,2.0,6.1,0.6,0.4,0.432,0.463,0.560,1.8,0.3,0.062,NaN
8149,Bam Adebayo,26,16,33.6,22.3,3.9,9.9,1.2,1.0,0.522,0.524,0.813,2.4,1.7,0.156,NaN
8150,Ochai Agbaji,23,20,20.1,5.8,1.1,2.7,0.6,0.6,0.458,0.578,0.667,1.5,0.5,0.057,NaN
8151,Santi Aldama,23,13,27.2,13.9,1.9,6.3,0.8,0.7,0.453,0.525,0.609,1.8,0.5,0.073,NaN
8152,Nickeil Alexander-Walker,25,19,23.5,6.7,2.7,2.2,0.9,0.7,0.410,0.521,0.455,1.9,0.8,0.086,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8444,Zion Williamson,23,18,31.2,22.7,4.9,5.8,1.2,0.3,0.569,0.572,0.664,2.6,1.7,0.147,NaN
8445,James Wiseman,22,10,11.1,5.4,0.3,3.2,0.1,0.3,0.649,0.649,0.545,1.8,0.2,0.065,NaN
8446,Christian Wood,28,21,20.2,7.0,0.8,5.9,0.2,0.6,0.443,0.504,0.689,1.8,0.3,0.038,NaN
8447,Trae Young,25,18,36.0,27.1,10.7,2.9,1.6,0.0,0.409,0.486,0.864,1.5,2.0,0.147,NaN


In [135]:
test_data.columns[1:15]

Index(['Age', 'G', 'MP', 'PTS', 'AST', 'TRB', 'STL', 'BLK', 'FG%', 'eFG%',
       'FT%', 'PF', 'WS', 'WS/48'],
      dtype='object')

In [137]:
features = test_data.columns[1:15]

test_features = test_data[features]
test_label = test_data['MVP']

X_train, X_test, y_train, y_test = train_test_split(test_features, test_label, test_size=0.2, random_state=42)

In [138]:
new_y_pred = model.predict(test_features)

new_accuracy = accuracy_score(test_label, new_y_pred)
print(f"Accuracy for LogisticRegression Model using 2023-24 NBA Data: {accuracy2:.2%}")

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values